<a href="https://colab.research.google.com/github/mmsamiei/world/blob/main/world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install Faker

In [37]:
from faker import Faker
import random
import datetime
from itertools import filterfalse
from dateutil import relativedelta


In [38]:
fake = Faker()

In [39]:
class Location:

  def __init__(self, map_size=50):
    self.x =  random.uniform(-map_size, map_size)
    self.y = random.uniform(-map_size, map_size)

  def distance(self, other_location):
    dx = other_location.x - self.x
    dy = other_location.y - self.y
    return (dx**2+dy**2)**0.5

In [40]:
random.choice([70,80,90,100])

70

In [124]:
class Person:
  
  def __init__(self, world):

    self.world = world
    self.gender = random.choices(['M', 'F'])
    self.first_name = fake.first_name_female() if self.gender == 'F' else fake.first_name_male()
    self.last_name = fake.last_name()
    self.birthday = self.world.date
    self.deathday = None
    self.location = Location()
    self.velocity = 10
    self.energy = 1
    self.hunger_thershold = random.choice([0.5,0.6,0.7,0.8,0.9,1])
    self.friends = []

  def __str__(self):
    x = round(self.location.x,2)
    y = round(self.location.y,2)
    age = relativedelta.relativedelta(self.world.date, self.birthday)

    return f"{self.first_name} {self.last_name}  at {x}, {y} energy: {self.energy}  age: {age.years, age.months}"
  

  def age(self):
    age = relativedelta.relativedelta(self.world.date, self.birthday)
    return age.years, age.months, age.days


  def can_eat(self):
    for food in self.world.foods:
      if self.location.distance(food.location) < self.world.distance_can_eat:
        return True, food
    return False, None
  
  def is_hungry(self):
    return self.energy < self.hunger_thershold

  def accept_friendship(self, person):
    return True

  def choose_action(self):
    
    can_eat = self.can_eat()
    if can_eat[0] is True:
      return 'eat', (self, can_eat[1]) 

    if self.is_hungry():
      day_to_death = self.energy / self.world.hunger_penalty
      dx = 0
      dy = 0
      for food in random.sample(self.world.foods[:], len(self.world.foods[:])):
        if self.location.distance(food.location)/self.velocity < day_to_death:
          dx = food.location.x - self.location.x
          dy = food.location.y - self.location.y
          length = (dx**2+dy**2)**0.5
          dx_norm = (dx / length) * (self.velocity ** 0.5)
          dy_norm = (dy / length) * (self.velocity ** 0.5)
          if dx**2 + dy**2 < dx_norm**2 + dy_norm**2:
            dir_x, dir_y = dx, dy
          else:
            dir_x, dir_y= dx_norm, dy_norm
          break
      return 'move', (self, dx, dy)
    

    other_persons = self.world.persons[:]
    other_persons.remove(self)
    for person in random.sample(other_persons, len(other_persons)):
      if self.location.distance(person.location) < 5 and person not in self.friends:
        return 'request_friendship', (self, person)
    
    return None, None

In [125]:
class Food:

  def __init__(self, world):
    self.world = world
    self.location = Location()
    self.produce_date = self.world.date

In [143]:
class World:

  def __init__(self, name):
    self.name = name
    self.persons = []
    self.dead_persons = []
    self.date = datetime.date(2018, 6, 1)
    self.foods = []
    self.hunger_penalty = 0.1
    self.distance_can_eat = 2
    self.every_day_foods = 10
    self.expire_day_foods = 5

  def simulate_day(self):
    self.date += datetime.timedelta(days=1)

    new_person = Person(world=self)
    self.persons.append(new_person)

    for i in range(self.every_day_foods):
      self.foods.append(Food(self))
    self.foods[:] = filterfalse(lambda f: (world.date - f.produce_date).days > self.expire_day_foods , self.foods)

    for person in self.persons[:]:
      self.simulate_person(person)
  

  def simulate_person(self, person):
    person.energy -= self.hunger_penalty
    action, args = person.choose_action()
    
    if action == 'eat':
      food = args[1]
      self.foods.remove(food)
      person.energy = 1
    elif action == 'move':
      dx, dy = args[1], args[2]
      person.location.x += dx
      person.location.y += dy
    elif action == 'request_friendship':
      person1, person2 = args[0], args[1]
      if person2.accept_friendship(person1):
        person1.friends.append(person2)
        person2.friends.append(person1)

    if person.energy <= 0 :
      self.persons.remove(person)
      self.dead_persons.append(person)
      person.deathday = self.date

  def simulate(self, until=3000):
    for i in range(until):
      self.simulate_day()
      #print(len(self.foods))
      print("persons ",len(self.persons), len(self.dead_persons))


In [144]:
world = World('mahdi_world')
world.simulate()

persons  1 0
persons  2 0
persons  3 0
persons  4 0
persons  5 0
persons  6 0
persons  7 0
persons  8 0
persons  9 0
persons  10 0
persons  11 0
persons  12 0
persons  13 0
persons  14 0
persons  15 0
persons  16 0
persons  17 0
persons  18 0
persons  19 0
persons  20 0
persons  21 0
persons  22 0
persons  23 0
persons  24 0
persons  25 0
persons  26 0
persons  27 0
persons  28 0
persons  29 0
persons  30 0
persons  31 0
persons  32 0
persons  33 0
persons  34 0
persons  35 0
persons  36 0
persons  37 0
persons  38 0
persons  39 0
persons  40 0
persons  41 0
persons  42 0
persons  43 0
persons  44 0
persons  45 0
persons  46 0
persons  47 0
persons  48 0
persons  49 0
persons  50 0
persons  49 2
persons  49 3
persons  49 4
persons  50 4
persons  51 4
persons  52 4
persons  53 4
persons  53 5
persons  54 5
persons  54 6
persons  54 7
persons  54 8
persons  55 8
persons  56 8
persons  56 9
persons  56 10
persons  55 12
persons  55 13
persons  56 13
persons  57 13
persons  55 16
persons  

In [145]:
for person in world.persons:
  print("\n")
  print(person.first_name, " " , person.last_name," has " ,len(person.friends) ,": ")
  for friend in person.friends:
    print(friend.first_name, " " , friend.last_name, friend.deathday)



Daniel   Brooks  has  93 : 
Peter   Brown 2018-07-24
Mike   Stephens 2018-08-20
Steven   Cruz 2020-02-01
Keith   Everett 2018-11-27
Charles   Simpson 2019-01-02
Matthew   Graham 2019-05-19
William   Douglas 2019-01-21
Timothy   Conner 2019-02-10
Jimmy   Caldwell 2019-08-22
Richard   Walsh 2019-04-25
Gabriel   Booth 2019-05-18
Reginald   King 2020-11-02
John   Young 2019-12-13
William   Long 2019-06-20
Erik   Hernandez 2019-11-25
Frank   Rose 2020-05-14
Robert   Cox 2019-09-28
Joseph   Parrish 2019-12-13
Todd   Garcia 2019-10-03
David   Guzman 2019-12-26
Matthew   Cunningham 2021-05-09
Jacob   Thompson 2020-02-12
Kerry   Turner 2020-07-19
Christopher   Hicks 2020-06-18
Bradley   Howell 2020-05-22
Robert   Brown 2020-06-27
Joshua   Willis 2020-09-22
Troy   Solomon 2020-09-30
Mark   Elliott 2020-10-25
Jesse   Abbott 2020-11-23
Frank   Powell 2021-03-10
James   Campbell 2020-10-28
Todd   Green 2021-01-22
Tony   Houston 2021-02-08
Robert   Wheeler 2021-04-09
Ryan   Butler 2021-05-16
Ethan